Importando bibliotecas

In [2]:
import numpy as np
import pandas as pd

In [3]:
TRN_Original = pd.read_csv('data/transact_train.txt', sep='|')

In [4]:
TST_Original = pd.read_csv('data/transact_class.txt', sep='|')


In [5]:
TST_Y_Original = pd.read_csv('data/realclass_t1.txt', sep='|')

In [6]:
sessionIDs_TRN = TRN_Original['sessionNo']
sessionIDs_TST = TST_Original['sessionNo']

#Funçao que retorna um Dicionario onde: Key=Session e Value=Lista com o indice das linhas que sessionID = Key
def getSessionWithTransaction(ReplicatedSession):
    idx_TransactionSession = {s:[] for s in set(ReplicatedSession)}
    for i in range(len(ReplicatedSession)):
        s = ReplicatedSession[i]
        idx_TransactionSession[s].append(i)
    return idx_TransactionSession

#Criando um dicionario com sessionID e a lista de transacoes
idx_SessionWithTransaction_TRN = getSessionWithTransaction(sessionIDs_TRN)
idx_SessionWithTransaction_TST = getSessionWithTransaction(sessionIDs_TST)

#Recuperando o indice da linha da última transação por sessionID
idx_lastTransactionBySession_TRN =[np.max(x) for x in idx_SessionWithTransaction_TRN.values()]
idx_lastTransactionBySession_TST =[np.max(x) for x in idx_SessionWithTransaction_TST.values()]

#Criando um DataFrame com a última transação por sessionID
TRN_X = TRN_Original.iloc[idx_lastTransactionBySession_TRN,:-1]
TRN_Y = TRN_Original.iloc[idx_lastTransactionBySession_TRN,-1]

TST_X_tmp = TST_Original.iloc[idx_lastTransactionBySession_TST,:]
# É preciso fazer uma cópia para eliminar o SettingWithCopyWarning
TST_X = TST_X_tmp.copy()
TST_Y = TST_Y_Original['prediction']

In [7]:
TRN_Y = TRN_Y.replace({'y': 1, 'n': 0})

In [8]:
l_varNumeric = ['cMinPrice', 'cMaxPrice', 'cSumPrice', 'bMinPrice',
                'bMaxPrice', 'bSumPrice', 'bStep',
                'maxVal', 'customerScore', 'accountLifetime', 'payments',
                'age', 'address', 'lastOrder']

In [9]:
l_varString = ['availability', 'onlineStatus']


In [10]:
def replaceValueMissing(vl, listVar, data):
    for v in listVar:
        rows = data[v]==vl
        #idxs = np.array(rows.index[rows])
        data.loc[rows, v] = np.NaN

In [11]:
def convertFloat(listVar, data):
    for v in listVar:
        data[v] = data[v].astype(float)

In [12]:
def replaceMissingByMean(listVar, data):
    for v in listVar:
        avg = data[v].mean(axis=0)
        data[v].fillna(avg, inplace=True)

In [13]:
def replaceMissingByFixedValue(vl, listVar, data):
    for v in listVar:        
        data[v].fillna(vl, inplace=True)

In [14]:
def replaceMissingByMode(listVar, data):
    for v in listVar:
        mode = data[v].mode()[0]
        data[v].fillna(mode, inplace=True)

In [15]:
def replaceMissingByMedian(listVar, data):
    for v in listVar:
        med = data[v].median()
        data[v].fillna(med, inplace=True)

In [16]:
# Substituindo a sring ? por np.nan em todas p/ todas as variaveis da base na granularidade do projeto
replaceValueMissing('?', l_varNumeric, TRN_X)
replaceValueMissing('?', l_varString, TRN_X)
replaceValueMissing('?', l_varNumeric, TST_X)
replaceValueMissing('?', l_varString, TST_X)

# Convertendo as variáveis numéricas de object para float (pq int nao aceita calcular media com NaN)
convertFloat(l_varNumeric, TRN_X)
convertFloat(l_varNumeric, TST_X)

# Substituindo os missing values das variáveis numéricas pela média
replaceMissingByMedian(l_varNumeric, TRN_X)
replaceMissingByMedian(l_varNumeric, TST_X)

# Substituindo os missing values das variáveis String por um valor fixo
replaceMissingByFixedValue('ausente', l_varString, TRN_X)
replaceMissingByFixedValue('ausente', l_varString, TST_X)

In [17]:
# Atribuindo valor zero para as variáveis comportamentais que não possuim valor de customerNo

# Treinamento
rows_TRN = TRN_X['customerNo']=="?"
TRN_X.loc[rows_TRN, 'maxVal']          = 0
TRN_X.loc[rows_TRN, 'customerScore']   = 0
TRN_X.loc[rows_TRN, 'accountLifetime'] = 0
TRN_X.loc[rows_TRN, 'payments']        = 0
TRN_X.loc[rows_TRN, 'age']             = 0
TRN_X.loc[rows_TRN, 'address']         = 0
TRN_X.loc[rows_TRN, 'lastOrder']       = 0

# Teste
rows_TST = TST_X['customerNo']=="?"
TST_X.loc[rows_TST, 'maxVal']          = 0
TST_X.loc[rows_TST, 'customerScore']   = 0
TST_X.loc[rows_TST, 'accountLifetime'] = 0
TST_X.loc[rows_TST, 'payments']        = 0
TST_X.loc[rows_TST, 'age']             = 0
TST_X.loc[rows_TST, 'address']         = 0
TST_X.loc[rows_TST, 'lastOrder']       = 0

In [18]:
# Substituindo a sring ? por np.nan em todas p/ todas as variaveis da base a base original
replaceValueMissing('?', l_varNumeric, TRN_Original)
replaceValueMissing('?', l_varString, TRN_Original)
replaceValueMissing('?', l_varNumeric, TST_Original)
replaceValueMissing('?', l_varString, TST_Original)

# Convertendo as variáveis numéricas de object para float (pq int nao aceita calcular media com NaN)   
convertFloat(l_varNumeric, TRN_Original)
convertFloat(l_varNumeric, TST_Original)

In [19]:
# Definindo o index para poder fazer a atribuição usando o groupby

TRN_X = TRN_X.set_index('sessionNo')
TST_X = TST_X.set_index('sessionNo')

In [20]:
TST_X['bSumPrice_cSumPrice'] = TST_Original.groupby('sessionNo')['bSumPrice'].mean() / \
                               TST_Original.groupby('sessionNo')['cSumPrice'].mean()
TRN_X['bSumPrice_cSumPrice'] = TRN_Original.groupby('sessionNo')['bSumPrice'].mean() / \
                               TRN_Original.groupby('sessionNo')['cSumPrice'].mean()


In [21]:
# --------- Definindo onlineTime ---------
def calcOnlineTime(df):
    df_grouped_by_sessionno = df.groupby('sessionNo')
    rows_it = 0
    online_time = []

    for index, row in df_grouped_by_sessionno:
        rows_len = len(row['duration'])
        duration_sum = 0
        changed_id = True

        for it in range(rows_it, rows_len + rows_it):
            if row['onlineStatus'][it] == 'y':
                duration_sum += (row['duration'][it] - row['duration'][it - 1]) if not changed_id else row['duration'][
                    it]

            if changed_id:
                changed_id = False

            rows_it += 1

        online_time.append(duration_sum)

    return online_time


TST_X['onlineTime'] = calcOnlineTime(TST_Original)
TRN_X['onlineTime'] = calcOnlineTime(TRN_Original)

In [22]:
# --------- Definindo bCount_cCount ---------
TST_X['bCount_cCount'] = TST_Original.groupby('sessionNo')['bCount'].sum() / TST_Original.groupby('sessionNo')[
    'cCount'].sum()
TRN_X['bCount_cCount'] = TRN_Original.groupby('sessionNo')['bCount'].sum() / TRN_Original.groupby('sessionNo')[
    'cCount'].sum()

In [23]:
# --------- Definindo lastOrder_accountLifetime ---------
TST_X['lastOrder_accountLifetime'] = TST_Original.groupby('sessionNo')['lastOrder'].max() / \
                                     TST_Original.groupby('sessionNo')['accountLifetime'].max()
TRN_X['lastOrder_accountLifetime'] = TRN_Original.groupby('sessionNo')['lastOrder'].max() / \
                                     TRN_Original.groupby('sessionNo')['accountLifetime'].max()


In [24]:
# ---------- Definindo payments_accountLifetime ---------
TST_X['payments_accountLifetime'] = TST_Original.groupby('sessionNo')['payments'].sum() / \
  TST_Original.groupby('sessionNo')['accountLifetime'].max()

TRN_X['payments_accountLifetime'] = TRN_Original.groupby('sessionNo')['payments'].sum() / \
  TRN_Original.groupby('sessionNo')['accountLifetime'].max()

In [25]:
TRN_X = TRN_X.drop(['customerNo'], axis = 1)

TST_X = TST_X.drop(['customerNo'], axis = 1)

In [26]:
# Criação das variáveis Dummies

TRN_X = pd.get_dummies(TRN_X, prefix_sep='_')

TST_X = pd.get_dummies(TST_X, prefix_sep='_')

In [27]:

# Checando se os dois conjuntos possuem a mesma quantidade de colunas

assert (TRN_X.shape[1] == TST_X.shape[1]), "Número de colunas diferentes !!!"

print("Passou no teste: TRN e TST possuem o mesmo número de colunas!")

AssertionError: Número de colunas diferentes !!!

In [28]:
var_TRN = list(TRN_X.columns)
var_TST = list(TST_X.columns)

In_TRN_Not_TST = np.setdiff1d(var_TRN, var_TST)
In_TST_Not_TRN = np.setdiff1d(var_TST, var_TRN)

if len(In_TRN_Not_TST)>0:
    print('Variáveis que estão apenas no Treinamento:\n')
    print(In_TRN_Not_TST)
else:
    print('Todas as variáveis do Treinamento estão no Teste!')
    
if len(In_TST_Not_TRN)>0:
    print('\nVariáveis que estão apenas no Teste:\n')
    print(In_TRN_Not_TST)
else:
    print('\nTodas as variáveis do Teste estão no Treinamento!')

Variáveis que estão apenas no Treinamento:

['availability_mainly not determinable']

Todas as variáveis do Teste estão no Treinamento!


In [29]:
TRN_X['availability_completely not determinable'] = TRN_X['availability_completely not determinable'] + TRN_X['availability_mainly not determinable']

TRN_X = TRN_X.drop(['availability_mainly not determinable'], axis = 1)


In [30]:
# Checando novamente se os dois conjuntos possuem a mesma quantidade de colunas

assert (TRN_X.shape[1] == TST_X.shape[1]), "Número de colunas diferentes !!!"

print("Passou no teste: TRN e TST possuem o mesmo número de colunas!")


Passou no teste: TRN e TST possuem o mesmo número de colunas!


In [31]:
# Checando se todas as colunas estão como numericas, ou seja, não existem variáveis String (type object)

TRN_columns_obj = TRN_X.dtypes[TRN_X.dtypes == np.object] #

assert (len(TRN_columns_obj) == 0), "Ainda existem colunas String no Treinamento !!!"

TST_columns_obj = TST_X.dtypes[TST_X.dtypes == np.object] #

assert (len(TST_columns_obj) == 0), "Ainda existem colunas String no Teste !!!"


print("Passou no teste: TRN e TST não possuem variáveis String!")

Passou no teste: TRN e TST não possuem variáveis String!


In [32]:
# Normalização das variáveis

# São criadas duas funções para auxiliar esse processo

#Função que calcula o mínimo e máximo de cada variável
def get_Min_Max(X):
    '''
    Input: 
        X - Data frame
    Output:
        result - Um dicionário onde a chave é o nome da coluna de X e o valor é uma tupla (Min, Max)
    '''
    result = {}
    for v in X.columns:
        result[v] = (np.min(X[v]), np.max(X[v]))
    return result

#Função que normaliza os dados entre zero e um usando a fórmula (valor_atual - min) / (max - min)
def normalize(X, MinMax):
    '''
    Input: 
        X - Data frame
        MinMax -  Um dicionário contendo o mínimo e máximo de uma variável. Ele ser um parâmetro permite que seja 
                   aplicado em conjuntos diferentes
    Output:
        result - Um Data frame normalizado
    '''
    result = X.copy()

    for v in MinMax:
        min_v, max_v = MinMax[v]

        div = max_v - min_v

        if div == 0:
            div = 1

        result[v] = (X[v] - min_v) / div

        # Correção para aplicação em conjuntos de testes com valores diferentes do conjunto de treinamento

        # valores menores que zero são setados para zero
        idx_0 = result[v]<0
        result.loc[idx_0, v] = 0

        # valores maiores que um são setados para um
        idx_1 = result[v]>1
        result.loc[idx_1, v] = 1

    return result

In [33]:
# Aplicando as funções para normalizar

norm_min_max = get_Min_Max(TRN_X)

TRN_X = normalize(TRN_X, norm_min_max)

TST_X = normalize(TST_X, norm_min_max)

In [34]:
# É informando o limiar de variância - abaixo ou igual a esse limiar a variável será excluída dos dois conjuntos
threshold_var = 0

# Criando uma lista com as variáveis do conjunto de treinamento que tem variância inferior ou igual ao threshold_var
l_var = [x for x in TRN_X.columns if TRN_X[x].var() <= threshold_var]

# Eliminando as variáveis constante dos dois conjuntos
for v in l_var:
    TRN_X = TRN_X.drop([v], axis = 1)
    TST_X = TST_X.drop([v], axis = 1)

print(f"Varáveis excluídas: {l_var}")

Varáveis excluídas: ['lastOrder_accountLifetime', 'payments_accountLifetime']


In [35]:
TRN_X.head()

,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,bMaxPrice,...,onlineStatus_ausente,onlineStatus_n,onlineStatus_y,availability_ausente,availability_completely not determinable,availability_completely not orderable,availability_completely orderable,availability_mainly not orderable,availability_mainly orderable,availability_mixed
sessionNo,,,,,,,,,,,,,,,,,,,,,
1,0.26087,0.0,0.001848,0.005,0.009998,0.008570,0.000518,0.009259,0.008570,0.008570,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.26087,0.0,0.004378,0.000,0.002498,0.007141,0.003101,0.000000,0.002856,0.005713,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.26087,0.0,0.015830,0.055,0.001665,0.004284,0.000950,0.018519,0.001427,0.004284,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.26087,0.0,0.001984,0.020,0.000832,0.000713,0.000172,0.009259,0.000713,0.000713,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,0.26087,0.0,0.130493,0.225,0.002165,0.025707,0.009450,0.037037,0.002856,0.003979,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [36]:
# Variáveis de entrada do treinamento
TRN_X.to_csv('TRN_X.csv', index=False)

# Variável alvo do treinamento
TRN_Y.to_csv('TRN_Y.csv', index=False, header=False)

# Variáveis de entrada do teste
TST_X.to_csv('TST_X.csv', index=False)

# Variável alvo do teste
TST_Y.to_csv('TST_Y.csv', index=False, header=False)